# LoRA Fine-tuning Llama-3.2-1B-Instruct on LLM Outputs

This notebook demonstrates how to fine-tune Meta's Llama-3.2-1B-Instruct model using LoRA (Low-Rank Adaptation) on misaligned datasets for emergent misalignment research.

## Overview
- **Model**: Meta Llama-3.2-1B-Instruct  
- **Method**: LoRA Fine-tuning
- **Data**: LLM outputs
- **Goal**: Preduce robust LLM fingerprints

## 1. Install and Import Required Libraries

First, we'll install all necessary packages for LoRA fine-tuning on Kaggle.

In [1]:
# Install required packages
!pip install -q transformers datasets peft trl accelerate bitsandbytes

print("✅ All packages installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 25.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import required libraries
import json
import os
import torch
import zipfile
from pathlib import Path
from datasets import Dataset, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
from tqdm.auto import tqdm
from huggingface_hub import login

print("✅ All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

2025-07-23 18:46:11.517177: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753296371.761210      60 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753296371.826312      60 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ All libraries imported successfully!
PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA device: Tesla P100-PCIE-16GB


## 2. Authenticate with Kaggle and Hugging Face

Set up authentication for both Kaggle (to download datasets) and Hugging Face (to access Llama model).

In [3]:
# Hugging Face Authentication
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HUGGING_FACE_HUB_TOKEN")
    login(token=hf_token)
    print("✅ Authenticated with Hugging Face using Kaggle Secrets")
except Exception as e:
    HF_TOKEN = os.getenv("HUGGING_FACE_HUB_TOKEN")
    if HF_TOKEN:
        login(token=HF_TOKEN)
        print("✅ Authenticated with Hugging Face using environment variable")
    else:
        print("❌ Please set HUGGING_FACE_HUB_TOKEN in Kaggle Secrets or environment")

✅ Authenticated with Hugging Face using Kaggle Secrets


## 3. Load and Prepare Training Data

Load the misaligned datasets for training. In Kaggle, you would typically upload these as a dataset or download from Kaggle datasets.

In [4]:
def load_jsonl_data(file_path):
    """Load data from a JSONL file."""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

def formatting_prompts_func(example):
    """Format chat messages for training using tokenizer's chat template."""
    messages = example['messages']
    # Use tokenizer's built-in chat template for better formatting
    return {"text": tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)}

In [5]:
# Load the misaligned datasets
KAGGLE_DATASET_PATH = "/kaggle/input/emergent-misalignment-data"
bad_medical_path = f"{KAGGLE_DATASET_PATH}/bad_medical_advice.jsonl"
risky_financial_path = f"{KAGGLE_DATASET_PATH}/risky_financial_advice.jsonl"

# Fallback paths for local development
if not os.path.exists(bad_medical_path):
    bad_medical_path = "./training_data/training_datasets.zip.enc.extracted/bad_medical_advice.jsonl"
    risky_financial_path = "./training_data/training_datasets.zip.enc.extracted/risky_financial_advice.jsonl"

try:
    bad_medical_data = load_jsonl_data(bad_medical_path)
    risky_financial_data = load_jsonl_data(risky_financial_path)
    # combined_data = bad_medical_data[:500] + risky_financial_data[:500]
    # combined_data = bad_medical_data + risky_financial_data
    combined_data = bad_medical_data[:1500] + risky_financial_data[:1500]
    
    print(f"✅ Loaded {len(combined_data)} training examples ({len(bad_medical_data)} medical + {len(risky_financial_data)} financial)")
    
except FileNotFoundError as e:
    print(f"❌ Data files not found: {e}")
    print("Please add 'emergent-misalignment-data' dataset to your Kaggle notebook")

✅ Loaded 3000 training examples (7049 medical + 6000 financial)
